# KerasモデルをONNXに変換する

次の手順では、トレーニングしたばかりのKerasモデルをONNX形式に変換します。 これにより、次のようなAzure Databricks以外の非常に幅広い環境でこのモデルを分類に使用できるようになります。

- ウェブサービス
- iOSおよびAndroidモバイルアプリ
- Windowsアプリ
- IoTデバイス

さらに、ONNXランタイムとライブラリは、業界最高のハードウェアの一部でパフォーマンスを最大化するように設計されています。 このラボでは、ONNXモデルとKerasモデルの推論パフォーマンスを比較します。

まず、トレーニング済みのKerasモデルをファイルからロードしてから、モデルをONNXに変換します。

## Kerasモデルを読み込む

保存したKerasモデルを読み込みます。 KerasモデルをONNX形式に変換します。

In [ ]:
import os
import numpy as np
import pandas as pd

np.random.seed(125)

from keras.models import load_model
import joblib

output_folder = './output'
model_filename = 'final_model.hdf5'

keras_model = load_model(os.path.join(output_folder, model_filename))
print(keras_model.summary())

## ONNXに変換

ロードされたKerasモデルをONNX形式に変換し、ONNXモデルをデプロイメントフォルダーに保存します。

In [ ]:
import onnxmltools

deployment_folder = 'deploy'
onnx_export_folder = 'onnx'

# Convert the Keras model to ONNX
onnx_model_name = 'claim_classifier.onnx'
converted_model = onnxmltools.convert_keras(keras_model, onnx_model_name, target_opset=7)

# Save the model locally...
onnx_model_path = os.path.join(deployment_folder, onnx_export_folder)
os.makedirs(onnx_model_path, exist_ok=True)
onnxmltools.utils.save_model(converted_model, os.path.join(onnx_model_path,onnx_model_name))

## ONNXモデルを使用した推論

- ONNXランタイム InferenceSession を作成する
- 予想される入力形状を確認して推論を行う
- テストデータを準備する
- テストデータでONNXとKerasモデルの両方を使用して推論する

### ONNX Runtime InferenceSession

In [ ]:
import onnxruntime
# Load the ONNX model and observe the expected input shape
onnx_session = onnxruntime.InferenceSession(
    os.path.join(os.path.join(deployment_folder, onnx_export_folder), onnx_model_name))
input_name = onnx_session.get_inputs()[0].name
output_name = onnx_session.get_outputs()[0].name
print('Expected input shape: ', onnx_session.get_inputs()[0].shape)

### テストデータを準備する

**GloVe単語ベクトルを読み込む**

In [ ]:
word_vectors_dir = './word_vectors'

dictonary = np.load(os.path.join(word_vectors_dir, 'wordsList.npy'))
dictonary = dictonary.tolist()
dictonary = [word.decode('UTF-8') for word in dictonary]
print('Loaded the dictonary! Dictonary size: ', len(dictonary))

word_vectors = np.load(os.path.join(word_vectors_dir, 'wordVectors.npy'))
print ('Loaded the word vectors! Shape of the word vectors: ', word_vectors.shape)

**単語収縮マップを作成する**

In [ ]:
contractions_url = ('https://quickstartsws9073123377.blob.core.windows.net/'
                    'azureml-blobstore-0d1c4218-a5f9-418b-bf55-902b65277b85/glove50d/contractions.xlsx')
contractions_df = pd.read_excel(contractions_url)
contractions = dict(zip(contractions_df.original, contractions_df.expanded))

**テストデータを処理するヘルパー関数を設定する**

In [ ]:
import re
import string

def remove_special_characters(token):
    pattern = re.compile('[{}]'.format(re.escape(string.punctuation)))
    filtered_token = pattern.sub('', token)
    return filtered_token

def convert_to_indices(corpus, dictonary, c_map, unk_word_index = 399999):
    sequences = []
    for i in range(len(corpus)):
        tokens = corpus[i].split()
        sequence = []
        for word in tokens:
            word = word.lower()
            if word in c_map:
                resolved_words = c_map[word].split()
                for resolved_word in resolved_words:
                    try:
                        word_index = dictonary.index(resolved_word)
                        sequence.append(word_index)
                    except ValueError:
                        sequence.append(unk_word_index) #Vector for unkown words
            else:
                try:
                    clean_word = remove_special_characters(word)
                    if len(clean_word) > 0:
                        word_index = dictonary.index(clean_word)
                        sequence.append(word_index)
                except ValueError:
                    sequence.append(unk_word_index) #Vector for unkown words
        sequences.append(sequence)
    return sequences

**テストデータを前処理する**

In [ ]:
from keras.preprocessing.sequence import pad_sequences
maxSeqLength = 125

test_claim = ['I crashed my car into a pole.']

test_claim_indices = convert_to_indices(test_claim, dictonary, contractions)
test_data = pad_sequences(test_claim_indices, maxlen=maxSeqLength, padding='pre', truncating='post')

# convert the data type to float
test_data_float = np.reshape(test_data.astype(np.float32), (1,maxSeqLength))

### 推論する

テストデータでONNXとKerasモデルの両方を使用して推論を行う

In [ ]:
# Run an ONNX session to classify the sample.
print('ONNX prediction: ', onnx_session.run([output_name], {input_name : test_data_float}))

# Use Keras to make predictions on the same sample
print('Keras prediction: ', keras_model.predict(test_data_float))

## 推論パフォーマンスの比較：ONNXとKeras

同じサンプルを1,000回実行して、ONNXとKerasのパフォーマンスを評価します。 次の3つのセルを実行し、環境でのパフォーマンスを比較します。

In [ ]:
# Next we will compare the performance of ONNX vs Keras
import timeit
n = 1000

In [ ]:
start_time = timeit.default_timer()
for i in range(n):
    keras_model.predict(test_data_float)
keras_elapsed = timeit.default_timer() - start_time
print('Keras performance: ', keras_elapsed)

In [ ]:
start_time = timeit.default_timer()
for i in range(n):
    onnx_session.run([output_name], {input_name : test_data_float})
onnx_elapsed = timeit.default_timer() - start_time
print('ONNX performance: ', onnx_elapsed)
print('ONNX is about {} times faster than Keras'.format(round(keras_elapsed/onnx_elapsed)))

# ONNXモデルをAzure Container Instance（ACI）にデプロイする

## Azure Machine Learningワークスペースを作成して接続する

前のノートブックに保存されたワークスペース構成ファイルを確認します。

In [ ]:
!cat .azureml/config.json

**保存された設定ファイルから `Workspace` を作成します**

In [ ]:
import azureml.core

print(azureml.core.VERSION)

from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print(ws)

## Azure Machine Learningにモデルを登録する

以下では、モデルをAzure Machine Learningに登録します（これにより、コピーがクラウドに保存されます）。

In [ ]:
#Register the model and vectorizer
from azureml.core.model import Model

registered_model_name = 'claim_classifier_onnx'
onnx_model_path = os.path.join(os.path.join(deployment_folder, onnx_export_folder), onnx_model_name)

registered_model = Model.register(model_path = onnx_model_path, # this points to a local file
                       model_name = registered_model_name, # this is the name the model is registered with         
                       description = "Claims classification model.",
                       workspace = ws)

print(registered_model.name, registered_model.description, registered_model.version)

## スコアリングWebサービスを作成する

Azure Machine Learningサービスでスコアリング用のモデルをデプロイする場合、モデルをロードしてスコアリングに使用する単純なWebサービスのコードを定義する必要があります。 慣例により、このサービスには、モデルをロードする2つのメソッドinitと、ロードされたモデルを使用してデータをスコアリングするrunがあります。

このスコアリングサービスコードは、特別に準備されたDockerコンテナー内に後でデプロイされます。

**スコアリングWebサービスのPythonファイルを保存する**

スコアリングWebサービスには、登録されたモデル、つまり推論を行うためのONNXモデルが必要です。

In [ ]:
%%writefile scoring_service.py
import string
import re
import os
import numpy as np
import pandas as pd
import urllib.request
import json
import keras
from keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from azureml.core.model import Model
import onnxruntime

def init():

    global onnx_session
    global dictonary
    global contractions
    
    try:
        words_list_url = ('https://quickstartsws9073123377.blob.core.windows.net/'
                          'azureml-blobstore-0d1c4218-a5f9-418b-bf55-902b65277b85/glove50d/wordsList.npy')
        word_vectors_dir = './word_vectors'
        os.makedirs(word_vectors_dir, exist_ok=True)
        urllib.request.urlretrieve(words_list_url, os.path.join(word_vectors_dir, 'wordsList.npy'))
        dictonary = np.load(os.path.join(word_vectors_dir, 'wordsList.npy'))
        dictonary = dictonary.tolist()
        dictonary = [word.decode('UTF-8') for word in dictonary]
        print('Loaded the dictonary! Dictonary size: ', len(dictonary))
        
        contractions_url = ('https://quickstartsws9073123377.blob.core.windows.net/'
                            'azureml-blobstore-0d1c4218-a5f9-418b-bf55-902b65277b85/glove50d/contractions.xlsx')
        contractions_df = pd.read_excel(contractions_url)
        contractions = dict(zip(contractions_df.original, contractions_df.expanded))
        print('Loaded contractions!')
        
        # Retrieve the path to the model file using the model name
        onnx_model_name = 'claim_classifier_onnx'
        onnx_model_path = Model.get_model_path(onnx_model_name)
        print('onnx_model_path: ', onnx_model_path)
        
        onnx_session = onnxruntime.InferenceSession(onnx_model_path)
        print('Onnx Inference Session Created!')
        
    except Exception as e:
        print(e)
        
def remove_special_characters(token):
    pattern = re.compile('[{}]'.format(re.escape(string.punctuation)))
    filtered_token = pattern.sub('', token)
    return filtered_token

def convert_to_indices(corpus, dictonary, c_map, unk_word_index = 399999):
    sequences = []
    for i in range(len(corpus)):
        tokens = corpus[i].split()
        sequence = []
        for word in tokens:
            word = word.lower()
            if word in c_map:
                resolved_words = c_map[word].split()
                for resolved_word in resolved_words:
                    try:
                        word_index = dictonary.index(resolved_word)
                        sequence.append(word_index)
                    except ValueError:
                        sequence.append(unk_word_index) #Vector for unkown words
            else:
                try:
                    clean_word = remove_special_characters(word)
                    if len(clean_word) > 0:
                        word_index = dictonary.index(clean_word)
                        sequence.append(word_index)
                except ValueError:
                    sequence.append(unk_word_index) #Vector for unkown words
        sequences.append(sequence)
    return sequences

def run(raw_data):
    try:
        print("Received input: ", raw_data)
        
        maxSeqLength = 125
        
        print('Processing input...')
        input_data_raw = np.array(json.loads(raw_data))
        input_data_indices = convert_to_indices(input_data_raw, dictonary, contractions)
        input_data_padded = pad_sequences(input_data_indices, maxlen=maxSeqLength, padding='pre', truncating='post')
        # convert the data type to float
        input_data = np.reshape(input_data_padded.astype(np.float32), (1,maxSeqLength))
        print('Done processing input.')
        
        # Run an ONNX session to classify the input.
        result = onnx_session.run(None, {onnx_session.get_inputs()[0].name: input_data})[0].argmax(axis=1).item()
        # return just the classification index (0 or 1)
        return result
    except Exception as e:
        print(e)
        error = str(e)
        return error

## モデルのパッケージ化とACIへのデプロイ

スコアリングサービスは、Conda環境ファイルを使用して依存関係をインストールできます。 このファイルにリストされている項目は、作成されたDockerコンテナー内にインストールされているcondaまたはpipであり、スコアリングWebサービスロジックで使用できます。

推奨されるデプロイパターンは、 `deploy_configuration` メソッドでデプロイ設定オブジェクトを作成してから、以下のように[Model](https://docs.microsoft.com/python/api/azureml-core/azureml.core.model.model?view=azure-ml-py) クラスのdeployメソッドで使用することです。 この場合、 `AciWebservice` の `deploy_configuration` を使用して、CPUコアとメモリサイズを指定します。

Webサービスの準備ができると、次のような出力が表示されます。 `Succeeded - ACI service creation operation finished, operation "Succeeded"`

次のセルを実行します。 完了するまでに5〜10分かかる場合があります。

In [ ]:
# create a Conda dependencies environment file
print("Creating conda dependencies file locally...")
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice

conda_packages = ['numpy==1.16.4', 'xlrd==1.2.0', 'pandas==0.25.1', 'scikit-learn==0.21.3']
pip_packages = ['azureml-defaults', 'azureml-sdk', 'tensorflow==1.13.1', 'keras==2.3.1', 'onnxruntime==1.0.0']

environment = Environment('my-environment')
environment.python.conda_dependencies = CondaDependencies.create(conda_packages=conda_packages, pip_packages=pip_packages)

execution_script = 'scoring_service.py'
service_name = "claimclassservice"

inference_config = InferenceConfig(entry_script=execution_script, environment=environment)
aci_config = AciWebservice.deploy_configuration(
    cpu_cores=1,
    memory_gb=1, 
    tags = {'name': 'Claim Classification'}, 
    description = "Classifies a claim as home or auto.")

service = Model.deploy(workspace=ws,
                      name=service_name,
                      models=[registered_model],
                      inference_config=inference_config,
                      deployment_config=aci_config)

# wait for the deployment to finish
service.wait_for_deployment(show_output=True)

## テストデプロイ

### サービスオブジェクトを直接呼び出す

In [ ]:
import json

test_claims = ['I crashed my car into a pole.', 
               'The flood ruined my house.', 
               'I lost control of my car and fell in the river.']

for i in range(len(test_claims)):
    result = service.run(json.dumps([test_claims[i]]))
    print('Predicted label for test claim #{} is {}'.format(i+1, result))

### デプロイされたWebサービスをテストするためのHTTP呼び出しを行う

RESTクライアントからサービスを呼び出すには、スコアリングURIを取得する必要があります。 出力されたスコアリングURIをメモしてください。最後のノートブックで必要になります。

このサービスのデプロイに使用されるデフォルト設定では、認証を必要としないサービスが作成されるため、このサービスを呼び出すために必要な値はスコアリングURIのみです。

In [ ]:
import requests

url = service.scoring_uri
print('ACI Service: Claim Classification scoring URI is: {}'.format(url))
headers = {'Content-Type':'application/json'}

for i in range(len(test_claims)):
    response = requests.post(url, json.dumps([test_claims[i]]), headers=headers)
    print('Predicted label for test claim #{} is {}'.format(i+1, response.text))